In [2]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [5]:
df = pd.read_csv('data/AAPL_ohlcv.csv', index_col='Date')

In [6]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2005-10-17,6.66,6.69,6.50,6.60,154208600
2005-10-18,6.57,6.66,6.44,6.45,152397000
2005-10-19,6.43,6.78,6.32,6.78,252170800
2005-10-20,6.72,6.97,6.71,6.93,339440500
2005-10-21,7.02,7.03,6.83,6.87,199181500


In [7]:
X = df.drop(columns=['Close'],axis=1)

In [9]:
X.head()

,Open,High,Low,Volume
Date,,,,
2005-10-17,6.66,6.69,6.50,154208600
2005-10-18,6.57,6.66,6.44,152397000
2005-10-19,6.43,6.78,6.32,252170800
2005-10-20,6.72,6.97,6.71,339440500
2005-10-21,7.02,7.03,6.83,199181500


In [10]:
y = df['Close']

In [11]:
y.head()

Date
2005-10-17    6.60
2005-10-18    6.45
2005-10-19    6.78
2005-10-20    6.93
2005-10-21    6.87
Name: Close, dtype: float64

In [12]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    [
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [13]:
X = preprocessor.fit_transform(X)

In [15]:
X

array([[-1.01939789, -1.0182869 , -1.02020152,  0.40153816],
       [-1.02053703, -1.01866217, -1.02096824,  0.38329487],
       [-1.02230901, -1.01716107, -1.02250169,  1.3880436 ],
       ...,
       [ 4.56514145,  4.52627781,  4.47376524, -0.6783267 ],
       [ 4.49059148,  4.56593188,  4.53459208, -0.73360642],
       [ 4.68968672,  4.70440839,  4.72013949, -0.62248808]])

In [16]:
X.shape

(3732, 4)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((2985, 4), (747, 4))

In [18]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.8663
- Mean Absolute Error: 0.4102
- R2 Score: 0.9999
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.7128
- Mean Absolute Error: 0.3646
- R2 Score: 0.9999


Lasso
Model performance for Training set
- Root Mean Squared Error: 1.8348
- Mean Absolute Error: 1.1600
- R2 Score: 0.9995
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.7705
- Mean Absolute Error: 1.1094
- R2 Score: 0.9995


Ridge
Model performance for Training set
- Root Mean Squared Error: 1.0934
- Mean Absolute Error: 0.5657
- R2 Score: 0.9998
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.9910
- Mean Absolute Error: 0.5263
- R2 Score: 0.9998


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 1.3790
- Mean Absolute Error: 0.8048
- R2 Score: 0.9997
-----------------------

c:\Users\devan\anaconda3\envs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.180e+03, tolerance: 1.867e+03
  model = cd_fast.enet_coordinate_descent(


Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 0.4491
- Mean Absolute Error: 0.2140
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.1526
- Mean Absolute Error: 0.5554
- R2 Score: 0.9998


XGBRegressor
Model performance for Training set
- Root Mean Squared Error: 0.4232
- Mean Absolute Error: 0.2933
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 3.4260
- Mean Absolute Error: 0.9329
- R2 Score: 0.9981


CatBoosting Regressor
Model performance for Training set
- Root Mean Squared Error: 0.9323
- Mean Absolute Error: 0.6249
- R2 Score: 0.9999
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 2.6301
- Mean Absolute Error: 0.8927
- R2 Score: 0.9989


AdaBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 5.1448
- Mean Absolute Error: 4.1398
- R2 Score: 0.99

In [20]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,0.999920
2,Ridge,0.999845
5,Random Forest Regressor,0.999790
4,Decision Tree,0.999625
1,Lasso,0.999504
3,K-Neighbors Regressor,0.999471
7,CatBoosting Regressor,0.998906
6,XGBRegressor,0.998143
8,AdaBoost Regressor,0.995320
